# Examen Final Automne 2020

Consignes:
- L'examen dure 3 heures. Les réponses peuvent être complétées sur le notebook ou hors ligne. Dans le dernier cas, veuillez déposer des photos ou des scans de l'examen sur studium. 15 minutes supplémentaires seront accordées pour le remise de l'examen.
- L'examen est à cours ouvert. Vous pouvez aussi vous inspirer des bloc-notes vus au cours pour compléter les parties Julia qui l'exigent.
- Si la réponse à une sous-question dépend de points précédents que vous n'avez pu résoudre, mais que vous avez l'idée du raisonnement pour cette sous-question, écrivez ce raisonnement.
- N'hésitez pas à utiliser Julia pour vous aider dans les calculs.

Nous allons importer les librairies suivantes pour faciliter les calculs et vérifier les réponses.

In [1]:
using LinearAlgebra
using JuMP
using Gurobi, Ipopt
using SparseArrays

Nous reprenons également les portions de codes suivantes. En particulier, nous utiliserons la routine de pivot.

In [2]:
function pivot!(A, i::Int, j::Int)
    (m, n) = size(A)
    A[i, :] = A[i, :] / A[i, j]
    for k in 1:m
        if k != i
            A[k, :] -= A[i, :]*A[k, j]
        end
    end
    return A
end

pivot! (generic function with 1 method)

In [3]:
# Heuristique proposée par Mehrotra
function genStart(A:: Matrix, b:: Vector, c:: Vector)
    x̅ = A'*((A*A')\b)
    λ̅ = (A*A')\A*c
    s̅ = c - A'*λ̅

    δx = max(-1.5*minimum(x̅),0)
    δs = max(-1.5*minimum(s̅),0)

    # δx et δs sont des scalaires, et doivent être ajoutés à chaque composante, c'est pourquoi nous utilisons .+
    num = dot(x̅ .+ δx, s̅ .+ δs)
    δx̅ = δx + num/(2*sum(s̅ .+ δs))
    δs̅ = δs + num/(2*sum(x̅ .+ δx))

    return (x̅ .+ δx̅, λ̅ , s̅ .+ δs̅)
end

function Mehrotra(A:: Matrix, b:: Vector, c:: Vector,
                  ε:: Float64, maxIter:: Int64 = 1000, sparseMode:: Bool = True)

    (m,n) = size(A)  # calcul la dimension de A
    
    if (sparseMode)
        temp = sparse(A)
        density = nnz(temp)/(m*n)
       
        # Si la densité de A est inférieure à 20%, nous travaillons avec une matrice creuse
        if (density < .2)
            A = sparse(A)
            b = sparse(b)
            c = sparse(c)
        end
    end

    # Calcul du point de départ
    res = genStart(A,b,c)
    x = res[1]
    λ = res[2]
    s = res[3]
    
    iter = 0

    # Nous utilisons le critère d'arrêt par défaut dans le test de boucle.
    # Le test de convergence se fera en interne, et nous pourrons alors quitter la fonction.
    while (iter < maxIter)
        iter += 1
        
        # Calcul des résidus
        rp = A*x-b
        rd = A'*λ + s - c
        rc = x .* s

        # μ est aligné sur le saut de dualité
        μ = dot(x,s)/n
        
        # Nous testons la qualité de la solution
        if (max(μ, norm(rd), norm(rp)) < ε)
            return (x,λ,s)
        end
        
        # Calcul de M = AXS^-1A' et du factory de Cholesky R de M (M=R'R)
        M = A*diagm(x./s)*A'
        R = cholesky(M).U

        # Calcul des termes de droite
        td = rp - A*((rc - x.*rd)./s)
        
        # Étape prédictrice
        Δλ_p = R\(R'\td)
        Δs_p = rd - A'*Δλ_p
        Δx_p = (rc - x.*Δs_p)./s
 
        # Calculs des longueur de pas potentiels les plus longs
        # Pas primal
        α_p = 1/maximum([1 ; Δx_p ./ x])
        # Pas dual
        α_d = 1/maximum([1 ; Δs_p ./ s])
        
        # Paramètre centralisant
        σ = (dot(x - α_p*Δx_p, s - α_d*Δs_p)/(n*μ))^3
        
        # Étape correctrice
        rc = rc .- σ*μ + Δx_p.*Δs_p
        td = rp - A*((rc - x.*rd)./s)
        
        Δλ_c = R\(R'\td)
        Δs_c = rd - A'*Δλ_c
        Δx_c = (rc - x.*Δs_c)./s
        
        η = max(0.995, 1-μ)
        
        # Pas primal
        α_p = η/maximum([η ; Δx_c./x])
        # Pas dual
        α_d = η/maximum([η ; Δs_c./s])
        
        # Mise à jour da la solution
        x -= α_p*Δx_c
        λ -= α_d*Δλ_c
        s -= α_d*Δs_c
    end
end

Mehrotra (generic function with 3 methods)

## Question 1 (20 pts)

Considérez le problème suivant
\begin{align*}
\max_x\ & z = 2 x_1 - 4 x_2 \\
\mbox{t.q } & x_1 - x_2 \leq 1 \\
& x_1, x_2 \geq 0.
\end{align*}

- Écrivez le problème dual.
- Utilisez les écarts complémentaires et la solution optimale du problème dual pour déterminer une solution optimale du primal.
- Pour quelles valeurs de $c_1$, le coefficient de $x_1$ dans l'objectif primal, le problème dual n’a pas de solution réalisable? Pour ces valeurs de $c_1$, qu’arrive-t-il pour le problème primal?

Le dual s'écrit simplement comme
\begin{align*}
\min\ & y \\
\mbox{t.q. } & 2 \leq y \leq 4
\end{align*}
On en déduit immédiatement que la solution optimale du dual est $y^* = 2$.

La première contrainte duale est active, mais pas la seconde, aussi $x_2^* = 0$. De plus, comme $y^* > 0$, la contrainte primale est active, et dès lors, $x_1^* = 1$.

Il suit immédiatement des points précédent que si $c_1 > 4$, le dual n'est pas réalisable comme nous aurions le programme
\begin{align*}
\min\ & y \\
\mbox{t.q. } & c_1 \leq y \leq 4
\end{align*}
Si le dual est non réalisable, le primal est non réalisable ou borné.
La contrainte du primal n'ayant pas changé, on en déduit que le problème primal est non borné, ce qui se vérifie aisément. En effet, le primal devient
\begin{align*}
\max\ & z = c_1x_1 - 4x_2 \\
\mbox{t.q. } & x_1 - x_2 \leq 1 \\
& x_1, x_2 \geq 0
\end{align*}
En prenant $x_2 = k$ et $x_1 = k+1$, avec $k \geq 0$ arbitraire, nous obtenons $x_1 - x_2 = 1$, i.e. la solution est réalisable, et $z = c_1(k+1)-4k = (c_1-4)k + 1$ tend vers $+\infty$ comme $k \rightarrow +\infty$.

## Question 2 (15 pts)

Utilisez la méthode duale du simplexe pour résoudre
\begin{align*}
\min_x \ & 2x_1 + x_2 \\
\mbox{t.q. } & 3x_1 + x_2 \geq 3 \\
& 4x_1 + 3x_2 \geq 6 \\
& x_1 + 2x_2 \leq 3 \\
& x_1,\ x_2 \geq 0.
\end{align*}

Créons et affichons le tableau du simplexe. Pour ce faire, nous devons ajoutez deux variables de surplus pour les deux premières contraintes, et une variable d'écart pour la dernière contrainte. Ceci donne, en multipliant les deux premières contraintes par -1:

In [4]:
A = [-3.0 -1 1 0 0; -4 -3 0 1 0; 1 2 0 0 1]
b = [-3;-6;3]
c = [2 1 0 0 0]
z0 = 0
S = [ A b ; c z0]

4×6 Array{Float64,2}:
 -3.0  -1.0  1.0  0.0  0.0  -3.0
 -4.0  -3.0  0.0  1.0  0.0  -6.0
  1.0   2.0  0.0  0.0  1.0   3.0
  2.0   1.0  0.0  0.0  0.0   0.0

Pivotons sur la première ligne. La comparaison entre la ligne des coûts réduits et la première ligne indique que l'élément pivot est alors l'élément (1,1).

In [5]:
pivot!(S,1,1)

4×6 Array{Float64,2}:
 1.0   0.333333  -0.333333  -0.0  -0.0   1.0
 0.0  -1.66667   -1.33333    1.0   0.0  -2.0
 0.0   1.66667    0.333333   0.0   1.0   2.0
 0.0   0.333333   0.666667   0.0   0.0  -2.0

Nous devons partir à présent de la seconde ligne. Le pivot est (2,2).

In [6]:
pivot!(S,2,2)

4×6 Array{Float64,2}:
  1.0  0.0  -0.6   0.2   0.0   0.6
 -0.0  1.0   0.8  -0.6  -0.0   1.2
  0.0  0.0  -1.0   1.0   1.0   0.0
  0.0  0.0   0.4   0.2   0.0  -2.4

La solution est réalisable et les coûts réduits non-négatifs. Dès lors, nous avons terminé. La solution optimale est $x_1^* = 0.6$, $x_2^* = 1.2$, pour une valeur optimale de 2.4.

## Question 3 (20 pts)

Résolvez à l'aide de l'algorithme branch and bound le problème
\begin{align*}
\max\ z\ & = 4x_1 - x_2 \\
\text{soumis à } & 7x_1 - 2x_2 \leq 14 \\
& x_2 \leq 3 \\
& 2x_1 - 2x_2 \leq 3 \\
& x_1 \geq 0, x_2 \geq 0 \\
& x_1, x_2 \text{ entiers.}
\end{align*}

Affichons la solution désirée.

In [7]:
m = Model(with_optimizer(Gurobi.Optimizer))

# Variables
@variable(m, x1 >= 0, Int)
@variable(m, x2 >= 0, Int)

# Fonction objectif
@objective(m, Max, 4x1 - x2)

# Contraintes
@constraint(m, constraint1, 7x1 -  2x2  <= 14)
@constraint(m, constraint2,  x2  <= 3)
@constraint(m, constraint3,  2x1 - 2x2  <= 3)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

Academic license - for non-commercial use only
Max 4 x1 - x2
Subject to
 constraint1 : 7 x1 - 2 x2 <= 14.0
 constraint2 : x2 <= 3.0
 constraint3 : 2 x1 - 2 x2 <= 3.0
 x1 >= 0.0
 x2 >= 0.0
 x1 integer
 x2 integer
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0xa0c6d6d7
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+01]
Found heuristic solution: objective 4.0000000
Presolve removed 3 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 7 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.000000000000e+00, best bound 7.000000000000e+00, gap 0.0000%

User-callback calls 38, time in user-callb

In [8]:
x = [ value(x1); value(x2) ]

2-element Array{Float64,1}:
 2.0
 1.0

Procédons à présent par branch & bound. La relaxation du problème donne

In [9]:
m = Model(with_optimizer(Gurobi.Optimizer))

# Variables
@variable(m, x1 >= 0)
@variable(m, x2 >= 0)

# Fonction objectif
@objective(m, Max, 4x1 - x2)

# Contraintes
@constraint(m, constraint1, 7x1 -  2x2  <= 14)
@constraint(m, constraint2,  x2  <= 3)
@constraint(m, constraint3,  2x1 - 2x2  <= 3)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

Academic license - for non-commercial use only
Max 4 x1 - x2
Subject to
 constraint1 : 7 x1 - 2 x2 <= 14.0
 constraint2 : x2 <= 3.0
 constraint3 : 2 x1 - 2 x2 <= 3.0
 x1 >= 0.0
 x2 >= 0.0
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0xa0c41c97
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4305000e+00   1.350000e-02   0.000000e+00      0s
       1    8.4285714e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds
Optimal objective  8.428571429e+00

User-callback calls 28, time in user-callback 0.00 sec


In [10]:
obj = []

Any[]

In [11]:
function sol_output()
    if (termination_status(m) == MOI.INFEASIBLE)
        println("Problème non réalisable")

        append!(obj, -Inf)
    else
        println("Solution optimale:")
        println("x1 = ", value(x1), " x2 = ", value(x2))
        println("Valeur optimale = ", objective_value(m))
        
        # On va enregistrer les valeurs objectifs trouvées.
        append!(obj, objective_value(m))
    end
end

sol_output (generic function with 1 method)

In [12]:
sol_output()

Solution optimale:
x1 = 2.857142857142857 x2 = 3.0
Valeur optimale = 8.428571428571429


1-element Array{Any,1}:
 8.428571428571429

Nous devons brancher sur x1.

In [13]:
set_upper_bound(x1, 2)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

sol_output()

Max 4 x1 - x2
Subject to
 constraint1 : 7 x1 - 2 x2 <= 14.0
 constraint2 : x2 <= 3.0
 constraint3 : 2 x1 - 2 x2 <= 3.0
 x1 >= 0.0
 x2 >= 0.0
 x1 <= 2.0
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 3 rows, 2 columns and 5 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [2e+00, 2e+00]
  RHS range        [3e+00, 1e+01]
       0    8.4285714e+00   8.571429e-01   0.000000e+00      0s
       2    7.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds
Optimal objective  7.500000000e+00

User-callback calls 46, time in user-callback 0.00 sec
Solution optimale:
x1 = 2.0 x2 = 0.5
Valeur optimale = 7.5


2-element Array{Any,1}:
 8.428571428571429
 7.5

In [14]:
delete_upper_bound(x1)
set_lower_bound(x1, 3)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

sol_output()

Max 4 x1 - x2
Subject to
 constraint1 : 7 x1 - 2 x2 <= 14.0
 constraint2 : x2 <= 3.0
 constraint3 : 2 x1 - 2 x2 <= 3.0
 x1 >= 3.0
 x2 >= 0.0
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 3 rows, 2 columns and 5 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [3e+00, 3e+00]
  RHS range        [3e+00, 1e+01]
       0    3.0000000e+30   1.625000e+30   3.000000e+00      0s

Solved in 2 iterations and 0.00 seconds
Infeasible model

User-callback calls 67, time in user-callback 0.00 sec
Problème non réalisable


3-element Array{Any,1}:
   8.428571428571429
   7.5
 -Inf

Le problème n'est pas réalisable pour $x_1 \geq 3$. Nous pouvons donc nous concentrer sur la $x_1 \leq 2$.

Nous branchons sur $x_2$.

In [15]:
delete_lower_bound(x1)
set_upper_bound(x1, 2)
set_upper_bound(x2, 0)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

sol_output()

Max 4 x1 - x2
Subject to
 constraint1 : 7 x1 - 2 x2 <= 14.0
 constraint2 : x2 <= 3.0
 constraint3 : 2 x1 - 2 x2 <= 3.0
 x2 >= 0.0
 x1 <= 2.0
 x2 <= 0.0
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 3 rows, 2 columns and 5 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [2e+00, 2e+00]
  RHS range        [3e+00, 1e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4285714e+00   3.857143e+00   0.000000e+00      0s
       2    6.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds
Optimal objective  6.000000000e+00

User-callback calls 86, time in user-callback 0.00 sec
Solution optimale:
x1 = 1.5 x2 = 0.0
Valeur optimale = 6.0


4-element Array{Any,1}:
   8.428571428571429
   7.5
 -Inf
   6.0

In [16]:
set_lower_bound(x2, 1)
set_upper_bound(x2, 3)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

sol_output()

Max 4 x1 - x2
Subject to
 constraint1 : 7 x1 - 2 x2 <= 14.0
 constraint2 : x2 <= 3.0
 constraint3 : 2 x1 - 2 x2 <= 3.0
 x2 >= 1.0
 x1 <= 2.0
 x2 <= 3.0
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 3 rows, 2 columns and 5 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 3e+00]
  RHS range        [3e+00, 1e+01]
       0    1.5000000e+01   3.937500e+00   0.000000e+00      0s
       1    7.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds
Optimal objective  7.000000000e+00

User-callback calls 103, time in user-callback 0.00 sec
Solution optimale:
x1 = 2.0 x2 = 1.0
Valeur optimale = 7.0


5-element Array{Any,1}:
   8.428571428571429
   7.5
 -Inf
   6.0
   7.0

Avec $x_2 >= 1$, la solution est entière pour une valeur objectif de 7.

Nous pouvons également élaguer la branche $x_2 <= 0$ comme la valeur objectif vaut alors 6, et il n'est pas possible d'atteindre une valeur plus grande en continuant à brancher.

Il n'y a plus de branche non élaguée aussi nous avons terminé. La solution optimale est bien $x_1 = 2$, $x_2 = 1$.

## Question 4 (10 pts)

Considérons le problème
\begin{align*}
\min\ & -4x_1 - 14x_2 \\
\text{soumis à } & 2x_1 +7x_2 \leq 21 \\
& 7x1 +2x2 \leq 21 \\
& x1 \geq 0, x2 \geq 0 .
\end{align*}
L'application de `Gurobi` donne la solution optimale x_1 = 0, x_2 = 3, tandis que le logiciel `Ipopt`, qui implémente une méthode de points intérieurs, donne la solution optimale x1 = 0.28155026155537355 x2 = 2.9195570979480405.

Pouvez-vous expliquer d'où provient cette différence?

Les deux logiciels donne pour solution duale optimale (-2, 0). Pouvez-vous vérifier que cette solution est effectivement optimale pour le problème dual, sans résoudre explicitement ce dernier problème?

Construisons le problème sous forme standard.

In [17]:
A = [ 2 7 1 0 ; 7 2 0 1]
b = [ 21; 21 ]
c = [ -4; -14; 0; 0 ]

4-element Array{Int64,1}:
  -4
 -14
   0
   0

La solution du simplexe est x_1 = 7/3, x_2 = 7/3. Si non appliquons l'algorithme de Merothra vu au cours, nous obtenons une solution différente:

In [18]:
x = Mehrotra(A,b,c,1e-10,10,true)[1]

println("x = ", x)
println("Ax = ", A*x)
println("z = c^Tx = ", dot(c,x))

x = [2.2446535127221634, 2.3586704249365242, 6.255453194316479e-16, 0.5700845610718075]
Ax = [20.999999999999996, 21.0]
z = c^Tx = -41.99999999999999


Appliquons Gurobi et IPopt.

In [19]:
m = Model(with_optimizer(Gurobi.Optimizer))

# Variables
@variable(m, x1 >= 0)
@variable(m, x2 >= 0)

# Fonction objectif
@objective(m, Min, -4x1 -14x2)

# Contraintes
@constraint(m, constraint1, 2x1 + 7x2 <= 21)
@constraint(m, constraint2, 7x1 + 2x2 <= 21)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

println("Optimal Solutions:")
println("x1 = ", value(x1), " x2 = ", value(x2))

Academic license - for non-commercial use only
Min -4 x1 - 14 x2
Subject to
 constraint1 : 2 x1 + 7 x2 <= 21.0
 constraint2 : 7 x1 + 2 x2 <= 21.0
 x1 >= 0.0
 x2 >= 0.0
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x9712c6d2
Coefficient statistics:
  Matrix range     [2e+00, 7e+00]
  Objective range  [4e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+01]
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.8000000e+31   2.250000e+30   1.800000e+01      0s
       1   -4.2000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds
Optimal objective -4.200000000e+01

User-callback calls 26, time in user-callback 0.00 sec
Optimal Solutions:
x1 = 0.0 x2 = 3.0


In [20]:
dual(constraint1)

-2.0

In [21]:
dual(constraint2)

0.0

In [22]:
m = Model(with_optimizer(Ipopt.Optimizer))

# Variables
@variable(m, x1 >= 0)
@variable(m, x2 >= 0)

# Fonction objectif
@objective(m, Min, -4x1 -14x2)

# Contraintes
@constraint(m, constraint1, 2x1 + 7x2 <= 21)
@constraint(m, constraint2, 7x1 + 2x2 <= 21)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

println("Optimal Solutions:")
println("x1 = ", value(x1), " x2 = ", value(x2))

Min -4 x1 - 14 x2
Subject to
 constraint1 : 2 x1 + 7 x2 <= 21.0
 constraint2 : 7 x1 + 2 x2 <= 21.0
 x1 >= 0.0
 x2 >= 0.0

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables w

In [23]:
sol_iptopt = [ value.(x1);  value.(x2) ]

2-element Array{Float64,1}:
 0.28155026155537355
 2.9195570979480405

In [24]:
dual(constraint1)

-2.0000000000432734

In [25]:
dual(constraint2)

-3.14697415402056e-10

La différence de solution vient du fait qu'un algorithme de point intérieur donne comme solution le centre analytique de la face optimale, tandis que l'algorithme du simplexe renvoit un point extrême. Or on remarque facilement ici que la solution n'est pas unique comme l'objectif est parallèle à la première contrainte, qui va former une face optimale. IPopt trouve une solution sur cette face optimale, tandis que Gurobi retourne un point extrême.

Problème dual:
\begin{align*}
\max \ & 21y_1 + 21y_2 \\
 & 2y_1 + 7y_2 \leq -4 \\
 & 7y_1 + 2y_2 \leq -14 \\
 & y_1 \leq 0, y_2 \leq 0.
\end{align*}

Les écarts de complémentarité donnent
\begin{align*}
& 2x_1 +7x_2 \leq 21 \\
& 7x1 +2x2 \leq 21 \\
& x1 \geq 0, x2 \geq 0 \\
 & 2y_1 + 7y_2 \leq -4 \\
 & 7y_1 + 2y_2 \leq -14 \\
 & y_1 \leq 0, y_2 \leq 0 \\
 & x_1(2y_1 + 7y_2 + 4) = 0 \\
 & x_2(7y_1 + 2y_2 +14) = 0
\end{align*}
Remarquons que si $x_1 \ne 0$, $x_2 \ne 0$, alors les deux équations duales doivent être actives, donnant la solution duale

In [26]:
D = [ 2 7 ; 7 2]
d = [ -4 ; -14 ]
D\d

2-element Array{Float64,1}:
 -2.0
  0.0

Cette solution satisfait la dualité forte et est donc optimale.
Si une des deux contraintes duales est inactive, la variable primale correspondante devrait valoir 0. Remarquons tout d'abord que les deux contraintes ne peuvent être inactives en même temps, car alors $x_1 = x_2 = 0$, mais la dualité forte ne tient plus.
Si $x_1 = 0$, $x_2 > 0$, alors $x_2 \leq 3$, et pour que la dualité forte tiennent, il faut $x_2 = 3$. Dès lors, $(0,3)$ est bien solution et l'objectif dual se réécrit $-42 - 6y_2$. La dualité forte implique $-14x_2 = -42-6y_2$, mais dès que $y_2 < 0$, on s'écarte de la valeur la plus basse possible, aussi $x_2 = 4$, $y_2 = 0$, $y_1 = -2$. Si $x_2 = 0$, $x_1 > 0$ et il faut $-4x_1 = -42-6y_1$. Nous ne pouvons atteindre 42 qu'avec $x_1 = 10.5$, $y_1 = 0$, mais alors la faisabilité primale est violée. La solution duale est donc unique.

Nous pouvons aussi le voir en utilisant la forme standard du primal
\begin{align*}
& 2x_1 +7x_2 + x_3 = 21 \\
& 7x1 +2x2 + x_4 = 21 \\
& x1 \geq 0, x2 \geq 0 \\
 & 2y_1 + 7y_2 \leq -4 \\
 & 7y_1 + 2y_2 \leq -14 \\
 & y_1 \leq 0 \\
 & y_2 \leq 0 \\
 & x_1(2y_1 + 7y_2 + 4) = 0 \\
 & x_2(7y_1 + 2y_2 +14) = 0 \\
 & x_3y_1 = 0 \\
 & x_4y_2 = 0
\end{align*}
Si $x_1 > 0$, $x_2 > 0$, alors $y_1 = -2$, $y_2 = 0$, et donc $x_3 = 0$. Par contre, on ne peut pas déterminer une valeur unique pour $x_1$ et $x_2$ comme l'objectif primal est parallèle à la première contrainte.
Si $x_1 = 0$, la valeur optimale est atteinte avec $x_2 = 3$, $x_3 = 0$, $x_4 = 15$. Dès lors $y_1 = -2$, $y_2 = 0$ et la dualité forte tient. On retrouve la même solution.
Alors $x_2 > 0$, la valeur optimale est atteinte pour $x_1 = 10.5$ mais ceci viole la faisabilité primale.

In [27]:
Δb = [0.7; 1]

2-element Array{Float64,1}:
 0.7
 1.0

Nous devons déterminer la base optimale donnée par Gurobi.

Sous forme standard, nous avons la solution $x_1 = 0$, $x_2 = 3$, $x_3 = 0$, $x_4 = 15$; autrement dit, la base est composée des deuxièmes et quatrièmes colonnes de la matrice $A$, avec

In [28]:
A = [ 2 7 1 0 ; 7 2 0 1]

2×4 Array{Int64,2}:
 2  7  1  0
 7  2  0  1

Dès lors,

In [29]:
B = A[:,[2,4]]

2×2 Array{Int64,2}:
 7  0
 2  1

Ainsi, la solution devient

In [30]:
Δx = B\Δb

2-element Array{Float64,1}:
 0.09999999999999999
 0.8

i.e. $x_1 = 0$, $x_2 = 3.1$, $x_3 = 0$, $x_4 = 15.8$. Vérifions-le avec Gurobi.

La variation de l'objectif est

In [31]:
Δz = dot(c[[2,4]], Δx)

-1.4

In [32]:
m = Model(with_optimizer(Gurobi.Optimizer))

# Variables
@variable(m, x1 >= 0)
@variable(m, x2 >= 0)

# Fonction objectif
@objective(m, Min, -4x1 -14x2)

# Contraintes
@constraint(m, constraint1, 2x1 + 7x2 <= 21.7)
@constraint(m, constraint2, 7x1 + 2x2 <= 22)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

println("Optimal Solutions:")
println("x1 = ", value(x1), " x2 = ", value(x2))

Academic license - for non-commercial use only
Min -4 x1 - 14 x2
Subject to
 constraint1 : 2 x1 + 7 x2 <= 21.7
 constraint2 : 7 x1 + 2 x2 <= 22.0
 x1 >= 0.0
 x2 >= 0.0
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xdd3e038c
Coefficient statistics:
  Matrix range     [2e+00, 7e+00]
  Objective range  [4e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+01]
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.8000000e+31   2.250000e+30   1.800000e+01      0s
       1   -4.3400000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds
Optimal objective -4.340000000e+01

User-callback calls 26, time in user-callback 0.00 sec
Optimal Solutions:
x1 = 0.0 x2 = 3.1


La solution diffère à nouveau de celle donnée par Ipopt, ce qui n'est pas surprenant vu que l'objectif et la première contrainte sont toujours parallèles.

In [33]:
m = Model(with_optimizer(Ipopt.Optimizer))

# Variables
@variable(m, x1 >= 0)
@variable(m, x2 >= 0)

# Fonction objectif
@objective(m, Min, -4x1 -14x2)

# Contraintes
@constraint(m, constraint1, 2x1 + 7x2 <= 21.7)
@constraint(m, constraint2, 7x1 + 2x2 <= 22)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

println("Optimal Solutions:")
println("x1 = ", value(x1), " x2 = ", value(x2))

Min -4 x1 - 14 x2
Subject to
 constraint1 : 2 x1 + 7 x2 <= 21.7
 constraint2 : 7 x1 + 2 x2 <= 22.0
 x1 >= 0.0
 x2 >= 0.0
This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints 

Modifions à présent les coûts dans la fonction objectif, en considérant comme nouvel objectif
$$
\min -3x_1 - 14 x_2.
$$
En supposant que la solution de base ne change pas, nous avons comme une variation nulle de l'objectif, comme $x_1 = 0$.

Évidemment, nous ne pouvons pas immédiatement le déduire avec Ipopt, mais remarquons à présent que les deux solveurs s'accordent.

In [34]:
m = Model(with_optimizer(Gurobi.Optimizer))

# Variables
@variable(m, x1 >= 0)
@variable(m, x2 >= 0)

# Fonction objectif
@objective(m, Min, -3x1 -14x2)

# Contraintes
@constraint(m, constraint1, 2x1 + 7x2 <= 21)
@constraint(m, constraint2, 7x1 + 2x2 <= 21)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

println("Optimal Solutions:")
println("x1 = ", value(x1), " x2 = ", value(x2))

Academic license - for non-commercial use only
Min -3 x1 - 14 x2
Subject to
 constraint1 : 2 x1 + 7 x2 <= 21.0
 constraint2 : 7 x1 + 2 x2 <= 21.0
 x1 >= 0.0
 x2 >= 0.0
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x9cd63989
Coefficient statistics:
  Matrix range     [2e+00, 7e+00]
  Objective range  [3e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+01]
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.7000000e+31   2.250000e+30   1.700000e+01      0s
       1   -4.2000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds
Optimal objective -4.200000000e+01

User-callback calls 26, time in user-callback 0.00 sec
Optimal Solutions:
x1 = 0.0 x2 = 3.0


In [35]:
m = Model(with_optimizer(Ipopt.Optimizer))

# Variables
@variable(m, x1 >= 0)
@variable(m, x2 >= 0)

# Fonction objectif
@objective(m, Min, -3x1 -14x2)

# Contraintes
@constraint(m, constraint1, 2x1 + 7x2 <= 21)
@constraint(m, constraint2, 7x1 + 2x2 <= 21)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

println("Optimal Solutions:")
println("x1 = ", value(x1), " x2 = ", value(x2))

Min -3 x1 - 14 x2
Subject to
 constraint1 : 2 x1 + 7 x2 <= 21.0
 constraint2 : 7 x1 + 2 x2 <= 21.0
 x1 >= 0.0
 x2 >= 0.0
This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints 

La solution est en effet à présent unique.

## Question 5 (15 points)

Considérons le problème
\begin{align*}
\min\ & 2x_1 + 6x_2 + 4x_3 + x_4 \\
\mbox{t.q. }
& x_1 + x_2 + x_3 + x_4 = 8 \\
& x_1 + x_2 = 3 \\
& 2x_3 + 3x_4 = 12 \\  
& x_1 \geq 0, x_2 \geq 0, x_3 \geq 0, x_4 \geq 0.
\end{align*}
Résolvez le problème à l'aide de l'algorithme primal-dual.

In [36]:
c = [ 2; 6; 4 ; 1]
A = [ 1 1 1 1 ;
      1 1 0 0 ;
      0 0 2 3 ]
b = [ 8; 3; 12 ]

(m,n) = size(A)

(3, 4)

Nous allons résoudre le problème directement afin de vérifier la solution que nous recherchons.

In [37]:
pr = Model(with_optimizer(Gurobi.Optimizer))

@variable(pr, x[1:n] >= 0)

for k = 1:m
    @constraint(pr, sum(A[k,i]*x[i] for i = 1:n) == b[k])
end

@objective(pr, Min, sum(x[i]*c[i] for i = 1:n))

print(pr)

Academic license - for non-commercial use only
Min 2 x[1] + 6 x[2] + 4 x[3] + x[4]
Subject to
 x[1] + x[2] + x[3] + x[4] == 8.0
 x[1] + x[2] == 3.0
 2 x[3] + 3 x[4] == 12.0
 x[1] >= 0.0
 x[2] >= 0.0
 x[3] >= 0.0
 x[4] >= 0.0


In [38]:
status = optimize!(pr)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 3 rows, 4 columns and 8 nonzeros
Model fingerprint: 0xa8b7b631
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+01]
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  2.000000000e+01

User-callback calls 27, time in user-callback 0.00 sec


In [39]:
value.(x)

4-element Array{Float64,1}:
 3.0
 0.0
 3.0
 2.0

Appliquons à présent l'algorithme primal-dual.

In [40]:
Art = [[ A I b ] ; [ zeros(1,n) ones(1,m) 0 ]]
Art[m+1,:] = Art[m+1,:] - sum(Art[i,:] for i = 1:m)
s = c - transpose(A)*zeros(m)
Art

4×8 Array{Float64,2}:
  1.0   1.0   1.0   1.0  1.0  0.0  0.0    8.0
  1.0   1.0   0.0   0.0  0.0  1.0  0.0    3.0
  0.0   0.0   2.0   3.0  0.0  0.0  1.0   12.0
 -2.0  -2.0  -3.0  -4.0  0.0  0.0  0.0  -23.0

In [41]:
println(s')

[2.0 6.0 4.0 1.0]


$P = \emptyset$

In [42]:
idx = findall(x -> x < 0, Art[m+1,1:n])
j = idx[argmin(s[idx]./-Art[m+1,idx])]

4

In [43]:
s = s-s[j]/Art[m+1,j]*Art[m+1,1:n]

4-element Array{Float64,1}:
 1.5
 5.5
 3.25
 0.0

$P = \{ 4 \}$

In [44]:
pivot!(Art, 3, 4)

4×8 Array{Float64,2}:
  1.0   1.0   0.333333  0.0  1.0  0.0  -0.333333   4.0
  1.0   1.0   0.0       0.0  0.0  1.0   0.0        3.0
  0.0   0.0   0.666667  1.0  0.0  0.0   0.333333   4.0
 -2.0  -2.0  -0.333333  0.0  0.0  0.0   1.33333   -7.0

In [45]:
idx = findall(x -> x < 0, Art[m+1,1:n])
j = idx[argmin(s[idx]./-Art[m+1,idx])]
s= s-s[j]/Art[m+1,j]*Art[m+1,1:n]

4-element Array{Float64,1}:
 0.0
 4.0
 3.0
 0.0

$P = \{ 1, 4 \}$

In [46]:
pivot!(Art, 2, 1)

4×8 Array{Float64,2}:
 0.0  0.0   0.333333  0.0  1.0  -1.0  -0.333333   1.0
 1.0  1.0   0.0       0.0  0.0   1.0   0.0        3.0
 0.0  0.0   0.666667  1.0  0.0   0.0   0.333333   4.0
 0.0  0.0  -0.333333  0.0  0.0   2.0   1.33333   -1.0

In [47]:
idx = findall(x -> x < 0, Art[m+1,1:n])
j = idx[argmin(s[idx]./-Art[m+1,idx])]
s= s-s[j]/Art[m+1,j]*Art[m+1,1:n]

4-element Array{Float64,1}:
 0.0
 4.0
 0.0
 0.0

$P = \{ 1, 3, 4 \}$

In [48]:
pivot!(Art, 1, 3)

4×8 Array{Float64,2}:
 0.0  0.0  1.0  0.0   3.0  -3.0  -1.0  3.0
 1.0  1.0  0.0  0.0   0.0   1.0   0.0  3.0
 0.0  0.0  0.0  1.0  -2.0   2.0   1.0  2.0
 0.0  0.0  0.0  0.0   1.0   1.0   1.0  2.22045e-16

Fin!

Solution: (3, 0, 3, 2)

## Question 6 (15 points)

Considérons le problème de programmation linéaire suivant:
\begin{align*}
\min_x \ & \sum_{j = 1}^n c_j x_j \\
\mbox{t.q. } & \sum_{j=1}^n a_jx_j \leq b \\
& x_j \geq 0,\ j = 1,\ldots,n,
\end{align*}
où nous retrouvons une seule contrainte. Supposons que $b > 0$, $a_j > 0$ et $c_j < 0$, $j = 1,\ldots,n$. De plus,
$$
\frac{c_1}{a_1} < \frac{c_2}{a_2} < \ldots < \frac{c_n}{a_n}.
$$

- Écrivez le dual de ce problème.
- Trouvez une solution optimale du problème dual (ceci se fait par inspection).
- Déterminez une solution optimale du problème primal en utilisant la solution optimale du dual et les écarts complémentaires.

Le dual s'écrit
\begin{align*}
\max_{\lambda} \ & b\lambda \\
\mbox{t.q. } & a_i\lambda \leq c_i,\ i = 1,\ldots,n \\
& \lambda \leq 0
\end{align*}

On en déduit immédiatement que
$$
\lambda \leq \min \left\{ \frac{c_i}{a_i},\ i = 1,\ldots,n \right\} = \frac{c_1}{a_1} < 0,
$$
en vertu des hypothèses de l'énoncé.
Comme le dual est un problème de maximisation, il suit directement que
$$
\lambda^* = \frac{c_1}{a_1},
$$
et la valeur optimale duale est
$$
\frac{c_1b}{a_1}.
$$

Comme, par hypothèse,
$$
\frac{c_1}{a_1} < \frac{c_j}{a_j},\ j = 2,\ldots,n
$$
les contraintes duales sont inactives, sauf la première, et des écarts de complémentarité, on tire $x_i^* = 0$, $i = 2,\ldots,n$.
La contrainte primale se réduit alors à
$$
a_1 x_1 \leq b,
$$
contrainte qui doit être active comme $\lambda \ne 0$.
On en tire la solution optimale primale
$$
x_1^* = \frac{b}{a_1}
$$
qui donne la valeur optimale
$$
\frac{c_1b}{a_1}.
$$
La propriété de dualité forte est vérifiée.
